##SETUP

In [7]:
import os
import time
import timeit
import pandas as pd
import random
import numpy as np
# Android ADB
from scrcpy import Client, const
from subprocess import check_output,Popen
import threading

# Image processing
from PIL import Image
import cv2
import pytesseract
# Notebook
from IPython.display import display,clear_output
from tqdm.notebook import trange, tqdm

import sys
# Start Scrcpy once per restart
if 'importlib' not in sys.modules:  
    start_scrcpy=True
    if start_scrcpy:
        proc = Popen(['C:\Programs\Scrcpy\scrcpy'], shell=True)
        time.sleep(6) # <-- sleep for 6''
        proc.terminate() # <-- terminate the process
# internal
import bot_core
import bot_perception

import importlib
importlib.reload(bot_core)
importlib.reload(bot_perception)


SLEEP_DELAY=0.1


bot = bot_core.Bot()

## RUN BOT

In [8]:
wait=0
combat = 0
grid_df =None
# Main loop
while(1):
    output = bot.battle_screen(True,True,floor=5)
    if output[1]=='fighting':
        wait = 0
        combat+=1
        if combat>50:
            bot.restart_RR()
            combat = 0
            continue
        battle_bar=trange(8)
        for i in battle_bar:
                time.sleep(0.2)
                # Spawn units
                bot.click(450,1360)
                # Upgrade units
                bot.mana_level([3,1,5,4],hero_power=True)
                # Try to merge units
                grid_df,unit_series,df_groups,info = bot.try_merge(prev_grid=grid_df)
                clear_output()
                display([f'{combat}, {i+1}/8 {output[1]}, {info}, \nAverage age: {grid_df['Age'].mean()}' ,grid_df,unit_series[0:5],df_groups])
    else:
        combat=0
        wait+=1
        if wait>40:
            print('RESTARTING')
            bot.restart_RR()
            wait=0
        print(output[1],'wait count:',wait)

['2, 2/8 fighting  need more units!',
     grid_id         unit  probability  rank  rank_error position   Age
 0    icon_0  chemist.png        264.0     1      6281.5   [0, 0]   2.0
 1    icon_1  chemist.png        218.0     1      5854.5   [0, 1]   8.0
 2    icon_2    empty.png        999.0     0         0.0   [0, 2]  14.0
 3    icon_3  chemist.png        211.0     1      5508.0   [0, 3]   6.0
 4    icon_4    empty.png        999.0     0         0.0   [0, 4]  29.0
 5    icon_5    empty.png        999.0     0         0.0   [1, 0]  14.0
 6    icon_6   zealot.png        508.0     3       141.0   [1, 1]   1.0
 7    icon_7   zealot.png        366.0     1      7545.5   [1, 2]   1.0
 8    icon_8    empty.png        999.0     0         0.0   [1, 3]  31.0
 9    icon_9    empty.png        999.0     0         0.0   [1, 4]  14.0
 10  icon_10    empty.png        999.0     0         0.0   [2, 0]  14.0
 11  icon_11    empty.png        999.0     0         0.0   [2, 1]  14.0
 12  icon_12   zealot.png 

# Play Game

In [2]:
# Restart game
bot.restart_RR()

## Quick restart

In [ ]:
# Drop co-op game
for i in range(30):
    bot.restart_RR(True)